In [1]:
import numpy as np
import pandas as pd

import tensorflow as tf

from tensorflow import feature_column
from tensorflow.keras import layers

In [2]:
#Vorlage: https://www.tensorflow.org/tutorials/structured_data/feature_columns
dataframe = pd.read_csv('adult_train.csv')

In [3]:
dataframe.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,class
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
4,18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,<=50K


In [4]:
dataframe['target'] = np.where(dataframe['class']=='>50K', 0, 1)

# Drop un-used columns.
dataframe = dataframe.drop(columns=['fnlwgt', 'class'])

In [6]:
#TODO die Fragezeichenwerte aus den Daten rausfiltern oder damit trainieren lassen?

In [7]:
# A utility method to create a tf.data dataset from a Pandas Dataframe
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
  dataframe = dataframe.copy()
  labels = dataframe.pop('target')
  ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
  if shuffle:
    ds = ds.shuffle(buffer_size=len(dataframe))
  ds = ds.batch(batch_size)
  return ds

In [8]:
dataframe.head()

,age,workclass,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,target
0,25,Private,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,1
1,38,Private,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,1
2,28,Local-gov,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,0
3,44,Private,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,0
4,18,?,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,1


In [9]:
feature_columns = []

# numeric cols
for header in ['age', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week']:
  feature_columns.append(feature_column.numeric_column(header))

# indicator_columns
indicator_column_names = ['workclass', 'education', 'marital-status', 'occupation', 'relationship',
                          'race', 'sex', 'native-country']
for col_name in indicator_column_names:
  categorical_column = feature_column.categorical_column_with_vocabulary_list(
      col_name, dataframe[col_name].unique())
  indicator_column = feature_column.indicator_column(categorical_column)
  feature_columns.append(indicator_column)

In [10]:
from sklearn.model_selection import train_test_split

train, val = train_test_split(dataframe, test_size=0.2)
feature_layer = tf.keras.layers.DenseFeatures(feature_columns)

batch_size = 32
train_ds = df_to_dataset(train, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)

print(len(train))

35166


In [10]:
tiny_model = tf.keras.Sequential([
    feature_layer,
    layers.Dense(16, activation='relu'),
    layers.Dense(1)
])

tiny_model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])


tiny_model.fit(train_ds, validation_data=val_ds, epochs=100)

Epoch 1/100
Consider rewriting this model with the Functional API.
Consider rewriting this model with the Functional API.
1089/1099 [============================>.] - ETA: 0s - loss: 2.3574 - accuracy: 0.7817WARNING:tensorflow:Layers in a Sequential model should only have a single input tensor, but we receive a <class 'dict'> input: {'age': <tf.Tensor 'ExpandDims:0' shape=(None, 1) dtype=int64>, 'workclass': <tf.Tensor 'ExpandDims_12:0' shape=(None, 1) dtype=string>, 'education': <tf.Tensor 'ExpandDims_3:0' shape=(None, 1) dtype=string>, 'education-num': <tf.Tensor 'ExpandDims_4:0' shape=(None, 1) dtype=int64>, 'marital-status': <tf.Tensor 'ExpandDims_6:0' shape=(None, 1) dtype=string>, 'occupation': <tf.Tensor 'ExpandDims_8:0' shape=(None, 1) dtype=string>, 'relationship': <tf.Tensor 'ExpandDims_10:0' shape=(None, 1) dtype=string>, 'race': <tf.Tensor 'ExpandDims_9:0' shape=(None, 1) dtype=string>, 'sex': <tf.Tensor 'ExpandDims_11:0' shape=(None, 1) dtype=string>, 'capital-gain': <tf.T

1099/1099 [==============================] - 2s 2ms/step - loss: 0.4212 - accuracy: 0.8427 - val_loss: 0.3269 - val_accuracy: 0.8493
Epoch 34/100
1099/1099 [==============================] - 2s 2ms/step - loss: 0.3825 - accuracy: 0.8438 - val_loss: 0.3491 - val_accuracy: 0.8466
Epoch 35/100
1099/1099 [==============================] - 3s 2ms/step - loss: 0.4044 - accuracy: 0.8426 - val_loss: 0.3291 - val_accuracy: 0.8402
Epoch 36/100
1099/1099 [==============================] - 2s 2ms/step - loss: 0.3871 - accuracy: 0.8439 - val_loss: 0.3759 - val_accuracy: 0.8318
Epoch 37/100
1099/1099 [==============================] - 2s 2ms/step - loss: 0.3877 - accuracy: 0.8460 - val_loss: 0.3171 - val_accuracy: 0.8541
Epoch 38/100
1099/1099 [==============================] - 2s 2ms/step - loss: 0.3703 - accuracy: 0.8450 - val_loss: 0.7389 - val_accuracy: 0.8154
Epoch 39/100
1099/1099 [==============================] - 2s 2ms/step - loss: 0.3671 - accuracy: 0.8459 - val_loss: 0.3727 - val_accuracy

1099/1099 [==============================] - 2s 2ms/step - loss: 0.3318 - accuracy: 0.8310 - val_loss: 0.3420 - val_accuracy: 0.8270
Epoch 90/100
1099/1099 [==============================] - 2s 2ms/step - loss: 0.3327 - accuracy: 0.8309 - val_loss: 0.3445 - val_accuracy: 0.8242
Epoch 91/100
1099/1099 [==============================] - 2s 2ms/step - loss: 0.3323 - accuracy: 0.8303 - val_loss: 0.3400 - val_accuracy: 0.8239
Epoch 92/100
1099/1099 [==============================] - 2s 2ms/step - loss: 0.3309 - accuracy: 0.8305 - val_loss: 0.3450 - val_accuracy: 0.8186
Epoch 93/100
1099/1099 [==============================] - 2s 2ms/step - loss: 0.3313 - accuracy: 0.8319 - val_loss: 0.3398 - val_accuracy: 0.8262
Epoch 94/100
1099/1099 [==============================] - 2s 2ms/step - loss: 0.3316 - accuracy: 0.8301 - val_loss: 0.3461 - val_accuracy: 0.8308
Epoch 95/100
1099/1099 [==============================] - 2s 2ms/step - loss: 0.3313 - accuracy: 0.8298 - val_loss: 0.3402 - val_accuracy

In [11]:
small_model = tf.keras.Sequential([
    feature_layer,
    layers.Dense(32, activation='relu'),
    layers.Dense(8, activation='relu'),
    layers.Dense(1)
])

small_model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])


small_model.fit(train_ds, validation_data=val_ds, epochs=100)

Epoch 1/100
Consider rewriting this model with the Functional API.
Consider rewriting this model with the Functional API.
1097/1099 [============================>.] - ETA: 0s - loss: 1.1849 - accuracy: 0.8119WARNING:tensorflow:Layers in a Sequential model should only have a single input tensor, but we receive a <class 'dict'> input: {'age': <tf.Tensor 'ExpandDims:0' shape=(None, 1) dtype=int64>, 'workclass': <tf.Tensor 'ExpandDims_12:0' shape=(None, 1) dtype=string>, 'education': <tf.Tensor 'ExpandDims_3:0' shape=(None, 1) dtype=string>, 'education-num': <tf.Tensor 'ExpandDims_4:0' shape=(None, 1) dtype=int64>, 'marital-status': <tf.Tensor 'ExpandDims_6:0' shape=(None, 1) dtype=string>, 'occupation': <tf.Tensor 'ExpandDims_8:0' shape=(None, 1) dtype=string>, 'relationship': <tf.Tensor 'ExpandDims_10:0' shape=(None, 1) dtype=string>, 'race': <tf.Tensor 'ExpandDims_9:0' shape=(None, 1) dtype=string>, 'sex': <tf.Tensor 'ExpandDims_11:0' shape=(None, 1) dtype=string>, 'capital-gain': <tf.T

1099/1099 [==============================] - 2s 2ms/step - loss: 0.3141 - accuracy: 0.8452 - val_loss: 0.3181 - val_accuracy: 0.8432
Epoch 34/100
1099/1099 [==============================] - 2s 2ms/step - loss: 0.3141 - accuracy: 0.8449 - val_loss: 0.3180 - val_accuracy: 0.8488
Epoch 35/100
1099/1099 [==============================] - 2s 2ms/step - loss: 0.3254 - accuracy: 0.8396 - val_loss: 0.3864 - val_accuracy: 0.7973
Epoch 36/100
1099/1099 [==============================] - 2s 2ms/step - loss: 0.3702 - accuracy: 0.8067 - val_loss: 0.3762 - val_accuracy: 0.8139
Epoch 37/100
1099/1099 [==============================] - 2s 2ms/step - loss: 0.3578 - accuracy: 0.8055 - val_loss: 0.3619 - val_accuracy: 0.7995
Epoch 38/100
1099/1099 [==============================] - 2s 2ms/step - loss: 0.3511 - accuracy: 0.8094 - val_loss: 0.3594 - val_accuracy: 0.8264
Epoch 39/100
1099/1099 [==============================] - 2s 2ms/step - loss: 0.3286 - accuracy: 0.8212 - val_loss: 0.3240 - val_accuracy

1099/1099 [==============================] - 2s 2ms/step - loss: 0.2974 - accuracy: 0.8504 - val_loss: 0.3133 - val_accuracy: 0.8384
Epoch 90/100
1099/1099 [==============================] - 2s 2ms/step - loss: 0.2968 - accuracy: 0.8528 - val_loss: 0.3160 - val_accuracy: 0.8409
Epoch 91/100
1099/1099 [==============================] - 2s 2ms/step - loss: 0.2963 - accuracy: 0.8509 - val_loss: 0.3191 - val_accuracy: 0.8395
Epoch 92/100
1099/1099 [==============================] - 2s 2ms/step - loss: 0.2971 - accuracy: 0.8528 - val_loss: 0.3185 - val_accuracy: 0.8393
Epoch 93/100
1099/1099 [==============================] - 2s 2ms/step - loss: 0.2961 - accuracy: 0.8526 - val_loss: 0.3225 - val_accuracy: 0.8474
Epoch 94/100
1099/1099 [==============================] - 2s 2ms/step - loss: 0.2956 - accuracy: 0.8519 - val_loss: 0.3125 - val_accuracy: 0.8372
Epoch 95/100
1099/1099 [==============================] - 2s 2ms/step - loss: 0.2951 - accuracy: 0.8530 - val_loss: 0.3119 - val_accuracy

In [14]:
medium_model = tf.keras.Sequential([
    feature_layer,
    layers.Dense(64, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(1)
])

medium_model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])


medium_model.fit(train_ds, validation_data=val_ds, epochs=100)

Epoch 1/100
Consider rewriting this model with the Functional API.
Consider rewriting this model with the Functional API.
1086/1099 [============================>.] - ETA: 0s - loss: 1.6416 - accuracy: 0.8018WARNING:tensorflow:Layers in a Sequential model should only have a single input tensor, but we receive a <class 'dict'> input: {'age': <tf.Tensor 'ExpandDims:0' shape=(None, 1) dtype=int64>, 'workclass': <tf.Tensor 'ExpandDims_12:0' shape=(None, 1) dtype=string>, 'education': <tf.Tensor 'ExpandDims_3:0' shape=(None, 1) dtype=string>, 'education-num': <tf.Tensor 'ExpandDims_4:0' shape=(None, 1) dtype=int64>, 'marital-status': <tf.Tensor 'ExpandDims_6:0' shape=(None, 1) dtype=string>, 'occupation': <tf.Tensor 'ExpandDims_8:0' shape=(None, 1) dtype=string>, 'relationship': <tf.Tensor 'ExpandDims_10:0' shape=(None, 1) dtype=string>, 'race': <tf.Tensor 'ExpandDims_9:0' shape=(None, 1) dtype=string>, 'sex': <tf.Tensor 'ExpandDims_11:0' shape=(None, 1) dtype=string>, 'capital-gain': <tf.T

1099/1099 [==============================] - 4s 3ms/step - loss: 0.3213 - accuracy: 0.8294 - val_loss: 0.3406 - val_accuracy: 0.8228
Epoch 34/100
1099/1099 [==============================] - 2s 2ms/step - loss: 0.3192 - accuracy: 0.8330 - val_loss: 0.3348 - val_accuracy: 0.8250
Epoch 35/100
1099/1099 [==============================] - 2s 2ms/step - loss: 0.3178 - accuracy: 0.8333 - val_loss: 0.3352 - val_accuracy: 0.8278
Epoch 36/100
1099/1099 [==============================] - 2s 2ms/step - loss: 0.3333 - accuracy: 0.8262 - val_loss: 0.3572 - val_accuracy: 0.8219
Epoch 37/100
1099/1099 [==============================] - 2s 2ms/step - loss: 0.3239 - accuracy: 0.8324 - val_loss: 0.3752 - val_accuracy: 0.8184
Epoch 38/100
1099/1099 [==============================] - 3s 2ms/step - loss: 0.3193 - accuracy: 0.8385 - val_loss: 0.3401 - val_accuracy: 0.8302
Epoch 39/100
1099/1099 [==============================] - 2s 2ms/step - loss: 0.3158 - accuracy: 0.8382 - val_loss: 0.3620 - val_accuracy

1099/1099 [==============================] - 2s 2ms/step - loss: 0.2717 - accuracy: 0.8608 - val_loss: 0.3947 - val_accuracy: 0.8343
Epoch 90/100
1099/1099 [==============================] - 2s 2ms/step - loss: 0.2727 - accuracy: 0.8593 - val_loss: 0.4004 - val_accuracy: 0.8368
Epoch 91/100
1099/1099 [==============================] - 2s 2ms/step - loss: 0.2697 - accuracy: 0.8627 - val_loss: 0.4046 - val_accuracy: 0.8261
Epoch 92/100
1099/1099 [==============================] - 2s 2ms/step - loss: 0.2691 - accuracy: 0.8613 - val_loss: 0.4177 - val_accuracy: 0.8401
Epoch 93/100
1099/1099 [==============================] - 2s 2ms/step - loss: 0.2678 - accuracy: 0.8616 - val_loss: 0.3885 - val_accuracy: 0.8384
Epoch 94/100
1099/1099 [==============================] - 2s 2ms/step - loss: 0.2668 - accuracy: 0.8634 - val_loss: 0.3741 - val_accuracy: 0.8326
Epoch 95/100
1099/1099 [==============================] - 2s 2ms/step - loss: 0.2670 - accuracy: 0.8636 - val_loss: 0.3882 - val_accuracy

In [15]:
large_model = tf.keras.Sequential([
    feature_layer,
    layers.Dense(512, activation='relu'),
    layers.Dense(512, activation='relu'),
    layers.Dense(512, activation='relu'),
    layers.Dense(512, activation='relu'),
    layers.Dense(1)
])

large_model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])


large_model.fit(train_ds, validation_data=val_ds, epochs=100)

Epoch 1/100
Consider rewriting this model with the Functional API.
Consider rewriting this model with the Functional API.
1098/1099 [============================>.] - ETA: 0s - loss: 1.8078 - accuracy: 0.7878WARNING:tensorflow:Layers in a Sequential model should only have a single input tensor, but we receive a <class 'dict'> input: {'age': <tf.Tensor 'ExpandDims:0' shape=(None, 1) dtype=int64>, 'workclass': <tf.Tensor 'ExpandDims_12:0' shape=(None, 1) dtype=string>, 'education': <tf.Tensor 'ExpandDims_3:0' shape=(None, 1) dtype=string>, 'education-num': <tf.Tensor 'ExpandDims_4:0' shape=(None, 1) dtype=int64>, 'marital-status': <tf.Tensor 'ExpandDims_6:0' shape=(None, 1) dtype=string>, 'occupation': <tf.Tensor 'ExpandDims_8:0' shape=(None, 1) dtype=string>, 'relationship': <tf.Tensor 'ExpandDims_10:0' shape=(None, 1) dtype=string>, 'race': <tf.Tensor 'ExpandDims_9:0' shape=(None, 1) dtype=string>, 'sex': <tf.Tensor 'ExpandDims_11:0' shape=(None, 1) dtype=string>, 'capital-gain': <tf.T

Epoch 33/100
1099/1099 [==============================] - 14s 12ms/step - loss: 0.3372 - accuracy: 0.8284 - val_loss: 0.3481 - val_accuracy: 0.8271
Epoch 34/100
1099/1099 [==============================] - 17s 16ms/step - loss: 0.3222 - accuracy: 0.8323 - val_loss: 0.3463 - val_accuracy: 0.8169
Epoch 35/100
1099/1099 [==============================] - 17s 16ms/step - loss: 0.3076 - accuracy: 0.8433 - val_loss: 0.3367 - val_accuracy: 0.8311
Epoch 36/100
1099/1099 [==============================] - 14s 13ms/step - loss: 0.3035 - accuracy: 0.8467 - val_loss: 0.3396 - val_accuracy: 0.8308
Epoch 37/100
1099/1099 [==============================] - 13s 12ms/step - loss: 0.3044 - accuracy: 0.8445 - val_loss: 0.3359 - val_accuracy: 0.8401
Epoch 38/100
1099/1099 [==============================] - 17s 15ms/step - loss: 0.3032 - accuracy: 0.8462 - val_loss: 0.3298 - val_accuracy: 0.8137
Epoch 39/100
1099/1099 [==============================] - 20s 19ms/step - loss: 0.3060 - accuracy: 0.8451 - val_

1099/1099 [==============================] - 14s 13ms/step - loss: 0.2724 - accuracy: 0.8609 - val_loss: 0.4743 - val_accuracy: 0.8327
Epoch 89/100
1099/1099 [==============================] - 16s 14ms/step - loss: 0.2725 - accuracy: 0.8611 - val_loss: 0.4127 - val_accuracy: 0.8377
Epoch 90/100
1099/1099 [==============================] - 17s 15ms/step - loss: 0.2711 - accuracy: 0.8614 - val_loss: 0.4779 - val_accuracy: 0.8391
Epoch 91/100
1099/1099 [==============================] - 14s 12ms/step - loss: 0.2693 - accuracy: 0.8609 - val_loss: 0.3682 - val_accuracy: 0.8279
Epoch 92/100
1099/1099 [==============================] - 15s 14ms/step - loss: 0.2788 - accuracy: 0.8597 - val_loss: 0.4300 - val_accuracy: 0.8400
Epoch 93/100
1099/1099 [==============================] - 15s 13ms/step - loss: 0.2691 - accuracy: 0.8613 - val_loss: 0.5489 - val_accuracy: 0.8304
Epoch 94/100
1099/1099 [==============================] - 16s 14ms/step - loss: 0.2685 - accuracy: 0.8609 - val_loss: 0.5426 

In [12]:
history = {}
optimizer = ['SGD', 'RMSprop', 'Adam', 'Adadelta', 'Adagrad', 'Adamax', 'Nadam', 'Ftrl']
#optimizer = ['Adam']
for x in optimizer:
    print(x)
    model = tf.keras.Sequential([
    feature_layer,
    layers.Dense(32, activation='relu'),
    layers.Dense(8, activation='relu'),
    layers.Dense(1)
    ])
    model.compile(optimizer=x,
                  loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                  metrics=['accuracy'])

    history[x] = model.fit(train_ds, validation_data=val_ds, epochs=100)
    
    print()
    

SGD
Epoch 1/100
Consider rewriting this model with the Functional API.
Consider rewriting this model with the Functional API.
1070/1099 [============================>.] - ETA: 0s - loss: 3.9732 - accuracy: 0.6316WARNING:tensorflow:Layers in a Sequential model should only have a single input tensor, but we receive a <class 'dict'> input: {'age': <tf.Tensor 'ExpandDims:0' shape=(None, 1) dtype=int64>, 'workclass': <tf.Tensor 'ExpandDims_12:0' shape=(None, 1) dtype=string>, 'education': <tf.Tensor 'ExpandDims_3:0' shape=(None, 1) dtype=string>, 'education-num': <tf.Tensor 'ExpandDims_4:0' shape=(None, 1) dtype=int64>, 'marital-status': <tf.Tensor 'ExpandDims_6:0' shape=(None, 1) dtype=string>, 'occupation': <tf.Tensor 'ExpandDims_8:0' shape=(None, 1) dtype=string>, 'relationship': <tf.Tensor 'ExpandDims_10:0' shape=(None, 1) dtype=string>, 'race': <tf.Tensor 'ExpandDims_9:0' shape=(None, 1) dtype=string>, 'sex': <tf.Tensor 'ExpandDims_11:0' shape=(None, 1) dtype=string>, 'capital-gain': <

1099/1099 [==============================] - 2s 2ms/step - loss: 0.5515 - accuracy: 0.7597 - val_loss: 0.5461 - val_accuracy: 0.7643
Epoch 34/100
1099/1099 [==============================] - 2s 2ms/step - loss: 0.5515 - accuracy: 0.7597 - val_loss: 0.5461 - val_accuracy: 0.7643
Epoch 35/100
1099/1099 [==============================] - 2s 2ms/step - loss: 0.5515 - accuracy: 0.7597 - val_loss: 0.5461 - val_accuracy: 0.7643
Epoch 36/100
1099/1099 [==============================] - 2s 2ms/step - loss: 0.5515 - accuracy: 0.7597 - val_loss: 0.5461 - val_accuracy: 0.7643
Epoch 37/100
1099/1099 [==============================] - 2s 2ms/step - loss: 0.5515 - accuracy: 0.7597 - val_loss: 0.5461 - val_accuracy: 0.7643
Epoch 38/100
1099/1099 [==============================] - 2s 2ms/step - loss: 0.5515 - accuracy: 0.7597 - val_loss: 0.5461 - val_accuracy: 0.7643
Epoch 39/100
1099/1099 [==============================] - 2s 2ms/step - loss: 0.5515 - accuracy: 0.7597 - val_loss: 0.5461 - val_accuracy

1099/1099 [==============================] - 2s 2ms/step - loss: 0.5515 - accuracy: 0.7597 - val_loss: 0.5461 - val_accuracy: 0.7643
Epoch 90/100
1099/1099 [==============================] - 2s 2ms/step - loss: 0.5515 - accuracy: 0.7597 - val_loss: 0.5461 - val_accuracy: 0.7643
Epoch 91/100
1099/1099 [==============================] - 2s 2ms/step - loss: 0.5515 - accuracy: 0.7597 - val_loss: 0.5461 - val_accuracy: 0.7643
Epoch 92/100
1099/1099 [==============================] - 2s 2ms/step - loss: 0.5515 - accuracy: 0.7597 - val_loss: 0.5461 - val_accuracy: 0.7643
Epoch 93/100
1099/1099 [==============================] - 2s 2ms/step - loss: 0.5515 - accuracy: 0.7597 - val_loss: 0.5461 - val_accuracy: 0.7643
Epoch 94/100
1099/1099 [==============================] - 2s 2ms/step - loss: 0.5515 - accuracy: 0.7597 - val_loss: 0.5461 - val_accuracy: 0.7643
Epoch 95/100
1099/1099 [==============================] - 2s 2ms/step - loss: 0.5515 - accuracy: 0.7597 - val_loss: 0.5461 - val_accuracy

1099/1099 [==============================] - 2s 2ms/step - loss: 0.3246 - accuracy: 0.8437 - val_loss: 0.3134 - val_accuracy: 0.8363
Epoch 22/100
1099/1099 [==============================] - 3s 3ms/step - loss: 0.3228 - accuracy: 0.8436 - val_loss: 0.3116 - val_accuracy: 0.8486
Epoch 23/100
1099/1099 [==============================] - 2s 2ms/step - loss: 0.3222 - accuracy: 0.8422 - val_loss: 0.3194 - val_accuracy: 0.8426
Epoch 24/100
1099/1099 [==============================] - 2s 2ms/step - loss: 0.3212 - accuracy: 0.8431 - val_loss: 0.3176 - val_accuracy: 0.8403
Epoch 25/100
1099/1099 [==============================] - 2s 2ms/step - loss: 0.3250 - accuracy: 0.8434 - val_loss: 0.3270 - val_accuracy: 0.8450
Epoch 26/100
1099/1099 [==============================] - 2s 2ms/step - loss: 0.3199 - accuracy: 0.8440 - val_loss: 0.3106 - val_accuracy: 0.8429
Epoch 27/100
1099/1099 [==============================] - 2s 2ms/step - loss: 0.3218 - accuracy: 0.8451 - val_loss: 0.3290 - val_accuracy

1099/1099 [==============================] - 2s 2ms/step - loss: 0.3129 - accuracy: 0.8490 - val_loss: 0.3137 - val_accuracy: 0.8505
Epoch 78/100
1099/1099 [==============================] - 2s 2ms/step - loss: 0.3161 - accuracy: 0.8485 - val_loss: 0.3082 - val_accuracy: 0.8445
Epoch 79/100
1099/1099 [==============================] - 2s 2ms/step - loss: 0.3176 - accuracy: 0.8482 - val_loss: 0.3117 - val_accuracy: 0.8418
Epoch 80/100
1099/1099 [==============================] - 2s 2ms/step - loss: 0.3215 - accuracy: 0.8493 - val_loss: 0.3162 - val_accuracy: 0.8403
Epoch 81/100
1099/1099 [==============================] - 2s 2ms/step - loss: 0.3180 - accuracy: 0.8483 - val_loss: 0.3139 - val_accuracy: 0.8501
Epoch 82/100
1099/1099 [==============================] - 2s 2ms/step - loss: 0.3144 - accuracy: 0.8490 - val_loss: 0.3128 - val_accuracy: 0.8413
Epoch 83/100
1099/1099 [==============================] - 2s 2ms/step - loss: 0.3147 - accuracy: 0.8490 - val_loss: 0.4400 - val_accuracy

...

1099/1099 [==============================] - 2s 2ms/step - loss: 0.3079 - accuracy: 0.8481 - val_loss: 0.3090 - val_accuracy: 0.8400
Epoch 98/100
1099/1099 [==============================] - 2s 2ms/step - loss: 0.3081 - accuracy: 0.8482 - val_loss: 0.3132 - val_accuracy: 0.8453
Epoch 99/100
1099/1099 [==============================] - 2s 2ms/step - loss: 0.3077 - accuracy: 0.8479 - val_loss: 0.3159 - val_accuracy: 0.8463
Epoch 100/100
1099/1099 [==============================] - 2s 2ms/step - loss: 0.3061 - accuracy: 0.8487 - val_loss: 0.3173 - val_accuracy: 0.8517

Nadam
Epoch 1/100
Consider rewriting this model with the Functional API.
Consider rewriting this model with the Functional API.
1077/1099 [============================>.] - ETA: 0s - loss: 1.1499 - accuracy: 0.7833WARNING:tensorflow:Layers in a Sequential model should only have a single input tensor, but we receive a <class 'dict'> input: {'age': <tf.Tensor 'ExpandDims:0' shape=(None, 1) dtype=int64>, 'workclass': <tf.Tenso

1099/1099 [==============================] - 2s 2ms/step - loss: 0.3143 - accuracy: 0.8462 - val_loss: 0.3130 - val_accuracy: 0.8510
Epoch 30/100
1099/1099 [==============================] - 2s 2ms/step - loss: 0.3198 - accuracy: 0.8456 - val_loss: 0.3176 - val_accuracy: 0.8488
Epoch 31/100
1099/1099 [==============================] - 3s 3ms/step - loss: 0.3162 - accuracy: 0.8463 - val_loss: 0.3135 - val_accuracy: 0.8323
Epoch 32/100
1099/1099 [==============================] - 2s 2ms/step - loss: 0.3152 - accuracy: 0.8458 - val_loss: 0.3113 - val_accuracy: 0.8362
Epoch 33/100
1099/1099 [==============================] - 2s 2ms/step - loss: 0.3133 - accuracy: 0.8477 - val_loss: 0.3154 - val_accuracy: 0.8294
Epoch 34/100
1099/1099 [==============================] - 2s 2ms/step - loss: 0.3138 - accuracy: 0.8470 - val_loss: 0.3202 - val_accuracy: 0.8536
Epoch 35/100
1099/1099 [==============================] - 2s 2ms/step - loss: 0.3164 - accuracy: 0.8467 - val_loss: 0.3184 - val_accuracy

1099/1099 [==============================] - 3s 2ms/step - loss: 0.3025 - accuracy: 0.8510 - val_loss: 0.3170 - val_accuracy: 0.8371
Epoch 86/100
1099/1099 [==============================] - 3s 2ms/step - loss: 0.3037 - accuracy: 0.8492 - val_loss: 0.3124 - val_accuracy: 0.8416
Epoch 87/100
1099/1099 [==============================] - 2s 2ms/step - loss: 0.3019 - accuracy: 0.8490 - val_loss: 0.3142 - val_accuracy: 0.8362
Epoch 88/100
1099/1099 [==============================] - 2s 2ms/step - loss: 0.3016 - accuracy: 0.8497 - val_loss: 0.3128 - val_accuracy: 0.8364
Epoch 89/100
1099/1099 [==============================] - 2s 2ms/step - loss: 0.3018 - accuracy: 0.8497 - val_loss: 0.3155 - val_accuracy: 0.8482
Epoch 90/100
1099/1099 [==============================] - 2s 2ms/step - loss: 0.3014 - accuracy: 0.8489 - val_loss: 0.3159 - val_accuracy: 0.8428
Epoch 91/100
1099/1099 [==============================] - 2s 2ms/step - loss: 0.3019 - accuracy: 0.8493 - val_loss: 0.3142 - val_accuracy

1099/1099 [==============================] - 2s 2ms/step - loss: 0.3968 - accuracy: 0.8307 - val_loss: 0.3932 - val_accuracy: 0.8259
Epoch 18/100
1099/1099 [==============================] - 2s 2ms/step - loss: 0.3935 - accuracy: 0.8332 - val_loss: 0.3879 - val_accuracy: 0.8328
Epoch 19/100
1099/1099 [==============================] - 2s 2ms/step - loss: 0.3906 - accuracy: 0.8344 - val_loss: 0.3850 - val_accuracy: 0.8313
Epoch 20/100
1099/1099 [==============================] - 2s 2ms/step - loss: 0.3873 - accuracy: 0.8359 - val_loss: 0.3824 - val_accuracy: 0.8338
Epoch 21/100
1099/1099 [==============================] - 2s 2ms/step - loss: 0.3844 - accuracy: 0.8375 - val_loss: 0.3822 - val_accuracy: 0.8280
Epoch 22/100
1099/1099 [==============================] - 2s 2ms/step - loss: 0.3819 - accuracy: 0.8379 - val_loss: 0.3777 - val_accuracy: 0.8337
Epoch 23/100
1099/1099 [==============================] - 2s 2ms/step - loss: 0.3791 - accuracy: 0.8381 - val_loss: 0.3754 - val_accuracy

1099/1099 [==============================] - 2s 2ms/step - loss: 0.3351 - accuracy: 0.8420 - val_loss: 0.3357 - val_accuracy: 0.8378
Epoch 74/100
1099/1099 [==============================] - 2s 2ms/step - loss: 0.3348 - accuracy: 0.8429 - val_loss: 0.3354 - val_accuracy: 0.8401
Epoch 75/100
1099/1099 [==============================] - 2s 2ms/step - loss: 0.3347 - accuracy: 0.8423 - val_loss: 0.3351 - val_accuracy: 0.8395
Epoch 76/100
1099/1099 [==============================] - 2s 2ms/step - loss: 0.3343 - accuracy: 0.8425 - val_loss: 0.3346 - val_accuracy: 0.8408
Epoch 77/100
1099/1099 [==============================] - 2s 2ms/step - loss: 0.3339 - accuracy: 0.8428 - val_loss: 0.3345 - val_accuracy: 0.8389
Epoch 78/100
1099/1099 [==============================] - 2s 2ms/step - loss: 0.3339 - accuracy: 0.8425 - val_loss: 0.3356 - val_accuracy: 0.8358
Epoch 79/100
1099/1099 [==============================] - 2s 2ms/step - loss: 0.3336 - accuracy: 0.8424 - val_loss: 0.3343 - val_accuracy

In [25]:
#TODO Vergleich optimierte Attribute gegen unveränderte
#TODO experimentieren wie das NN verbessert werden kann: verschiedene Netzwerkgrößen, Epochgröße, Optimizer, activation Funktion
#TODO save model
#TODO nochmal anschauen wie die Ausgaben aufbereitet werden können: print(history['Adam'].history)

{'loss': [0.34823086857795715, 0.34771963953971863, 0.3473048806190491, 0.34724071621894836, 0.34695565700531006, 0.34631097316741943, 0.3462755084037781, 0.3459360897541046, 0.34582868218421936, 0.345500111579895], 'accuracy': [0.8157026767730713, 0.8153045773506165, 0.8170676231384277, 0.8163851499557495, 0.8161576390266418, 0.8162714242935181, 0.8171813488006592, 0.816641092300415, 0.817750096321106, 0.8166979551315308], 'val_loss': [0.3582903742790222, 0.3559153079986572, 0.35654234886169434, 0.3558173179626465, 0.3566613793373108, 0.355616956949234, 0.35621172189712524, 0.3552054464817047, 0.3565405607223511, 0.35931962728500366], 'val_accuracy': [0.8061874508857727, 0.8157415986061096, 0.8135805130004883, 0.8134667873382568, 0.8037989139556885, 0.8063011765480042, 0.8097133636474609, 0.8081210255622864, 0.8167652487754822, 0.8107370138168335]}


In [ ]:
#load testdata 
dataframe_test = pd.read_csv('adult_test.csv')

In [ ]:
#prepare testdata
dataframe_test['target'] = np.where(dataframe_test['class']=='>50K', 0, 1)
dataframe_test = dataframe_test.drop(columns=['fnlwgt', 'class'])
test_ds = df_to_dataset(dataframe_test, shuffle=False, batch_size=batch_size)

In [ ]:
#Für Abschluss Validierung an ungesehenen Datensatz

#loss, accuracy = model.evaluate(test_ds)
#print("Accuracy", accuracy)